# Dokumentácia k projektu pre predmet Neurónové siete
## Klasifikácia malých obrázkov pomocou neurónových sietí
### Klasifikácia do 10 a 100 tried

Cvičenie: Pondelok 12:00, Michal Farkaš
Filip Jandura, Dennis Sobolev

## Opis architektúry modelu
Model png,
odkaz na github - inšpirácia
Prerobený z lowlevel Tensorflow do Keras

Originál vs my
Obmena SimpleNet_vanila na SimpleNet_Residual - pridané reziduálne bloky
Pridané Augmentácie a Normalizácia dát


Description of the neural model architecture.
Description of the training routine.
Description of the experiments you conducted.
The results of these experiments and their analysis

# Augmentácie a Normalizácia dát
# Opis trénovania sietí

Či už SimpleNet vanila alebo Simplenet s Reziduálnymi blokmi, oba sa trénovali na rovnako upravených dátach z datasetu. Síce CIFAR-10 obsahuje pre 10 tried postačujúce množstvo trénovacích dát pre každú triedu (1000 images per class) v prípade CIFAR-100 je to len 100 obrazov na triedu, pričom tried v datasete je 100.

### Na oba datasety bola na začiatku aplikovaná *Normalizácia*.
 - ZCA whitening - https://www.kdnuggets.com/2018/10/preprocessing-deep-learning-covariance-matrix-image-whitening.html/3 
   - Normalizácia, ktorej parametre sa prepočítavajú štatisticky cez väčšie množstvo snímok - idálne celý dataset (v našom prípade 2000 v prvých experimentoch a potom 5000 obrázkov v ďalších experimentoc), pomocou ktorých sa vypočítajú parametre gradientu pozadia a vďaka operácií sa zvýraznia hrany objektov v porovnaní s pozadím/vnútorným ofarbením. Takto sa na obrazoch zachovávajú najdôležitejšie príznaky, ktoré neurónové siete vedia ľahšie extrahovať pre klasifikáciu.
 - Normalizácia škálovaním - "Zero Mean Unit Variance"
   - Hodnoty každého obrazu boli preškálované tak, že Priemerná hodnota všetkých pixelov všetkých kanálov = 0 (Zero Mean) a smerodajná odchýlka = 1 (Unit Variance). Viaceré práce dokazujú, že takáto normalizácia a distribúcia hodnôt napomáha gradientu neurónových sietí a prispieva k rýchlejšiemu učeniu. Pri správnom výbere Batch Normalizácie sa dáta udržiavajú práve takto škále

### Augmentácie dát
 Spoločné augmentácie
 - Rozšírenie RGB kanálov o ich prepočet do HSV farebnej škály. Okrem štandardných kanálov, sme pridali ďalšie tri kanály kódovania farby a teda vstupné dáta sú rozmerovo [32,32,6]. Výhoda HSV je v reprezentácií farby - celá farebná škála je vyjadrená kanálom HUE, Saturácia vyjadruje sýtosť a teda na tomto osobitnom kanáli sú lepšie viditeľné hrany objektov s rôznymi farbami - množstvo šedej farby v pixeli, Value je úroveň svetlosti pixelu.

CIFAR-10
 Tým že dataset sám o sebe obsahuje postačujúce množsto obrázkov, nebolo potrebné rozširovať dataset o veľké množstvo nových samplov. Použili sme len augmentácie, ktoré dávali zmysel podľa domény a obsahu obrázkov.
  - Pridanie zrkadlového objektu - Flipnutie obrazu pozdĺž Y osi - vertikálne.

CIFAR-100
 Dataset si vyžadoval rozsiahlejšie augmentácie
  - Pridanie zrkadlového odrazu - Flipnutie obrazu pozdĺž Y osi - vertikálne.
  - Pridanie zrkadlového odrazu - Flipnutie obrazu pozdĺž X osi - horizontálne. Na všetky doteraz vytvorené dáta. (Dataset obsahuje zvieratá, lietadlá... = objekty, ktoré sa môžu nachádzať vo svete aj "dole-hlavou".)
  - Pridanie rotácie o 90° na všetky doteraz vytvorené dáta - čím vznikli aj akoby rotácie o 180° a 270°

Následne sú dáta uložené a pri každom ďalšom spustení sa načítajú už predspracované dáta.
Pri CIFAR-100 je problém ich načítať do RAM (spolu 400000 obrázkov), preto sa načítavajú postupne vždy po 1 uloženom súbore a posledný súbor (40000 obrázkov) sa používa na Validáciu.


## Experimenty a Vyhodnotenie

Vykonávali sme experimenty na sieťach SimpleNet (Vanila) a SimpleNet_Residual a porovnávali sme ich efektívnosť.

1. Experimenty s hodnotou learning_rate
2. Experimenty s veľkostou batch
3. Dĺžka trénovania

V našich experimentoch sme trénovali siete s rôznymi veľkosťami vstupných batchov a regularizovali sme učiacu sa chybu pomocou hodnôt learning_rate. Čím väčší batch, tým sme si mohli dovoliť väčší learning rate, pretože chyba, ktorá sa počítala na batchi sa zovšeobecnila na väčšom množstve vstupných dát - batch.
  Pokiaľ boli learning_rate a batch_size nevyvážené, stávalo sa, že gradient/loss priveľmi kolísal a trénovanie bolo nestále. 
Pokiaľ sme odhadli správnu hodnotu tréning bol rýchly a pomerne stabilný.

Najväčšie rozdiely medzi trénovaním SimpleNet_Vanila a s Reziduálnymi blokmi sme zaznamenali v rýchlosti trénovania, stabilite trénovania a preučeniu sa na trénovacích dátach. Vanila sieť sa trénovala 59 epôch, kedy bola zastavená Early Stoppingom a dosiahla necelých 30% Accuracy. Sieť s reziduálnymi blokmi nebola zastavená počas žiadneho trénovania a bola schopná dosiahnuť aj po 12 epochách úspešnosť 78%. Pri tomto pozorovaní vidíme radikálne rozdiely a výhody využitia Reziduálnych blokov. 

Síce uvádzame, že sme po 59. epochách nedosiahli článkom uvádzanú úspešnosť 95.1% avšak ich tréning trval 200 epôch.
V porovnaní s Reziduálnymi sieťami, sme sa k tomuto výsledku priblížili už po menej ako 20 epochách. Dotrénovanie na takú úroveň by si vyžadovalo postupné znižovanie learning rate - tzv. fine tuning.
 
Najlepšie výsledky:
-- lr=0.01, batch 32   - SimpleNet_v 50.E acc: 0.14 SimpleNet_r 30.E acc: 0.63
-- lr=0.01, batch 64   - SimpleNet_v 50.E acc: 0.24 SimpleNet_r 20.E acc: 0.68
-- lr=0.01, batch 128  - SimpleNet_v 51.E acc: 0.38 SimpleNet_r 20.E acc: 0.83
-- lr=0.001, batch 64  - SimpleNet_v 51.E acc: 0.27 SimpleNet_r 47.E acc: 0.58
-- lr=0.001, batch 128 - SimpleNet_v 59.E acc: 0.33 SimpleNet_r 48.E acc: 0.62

Keďže ide o multiclass klasifikáciu museli sme vyberať metriky vhodné pre tento typ klasifikácie
Záverečné testovanie na najlepšej SimpleNet_Residual so všetkými implementovanými metrikami:
Categorical Accuracy Top 1 -- 0.771
Categorical Accuracy Top 2 -- 0.905
Categorical Accuracy Top 3 -- 0.953
Categorical Accuracy Top 5 -- 98% úspešnosť - priveľmi jednoduché, až polovica výsledkov
ROC krivka - určuje ako ľahko je možné oddeliť pozitívnu triedu voči negatívnym - s akým fixným tresholdom na softmax confidence z výstupoch z NN (logits), by sa dalo s určitosťou povedať, že si sieť bola istá svojim rozhodnutím - Vieme určiť úroveň natrénovania siete.
PR krivka - určuje pomer medzi presnosťou - úspešnosť klasifikácie vs. celkové množstvo hlasovaní za kladné triedy.

Pre vyhodnotenie úspešnosti a neúspešnosti a najmä zistenie na ktorých vstupoch sa sieť najviac mýlila, sme po teste - predikovali výstupy pre všetky triedy. Následne sme z každej triedy vybrali K (5) chybných predikcií, ktoré sme logovali a vizualizovali v tensorboarde. Následne sme analyzovali možné dôvody, prečo sa sieť na týchto konkrétnych výstupoch pomýlila.
Triedy sme vizualizovali v rozdelení "Ground truth" a "False prediction" triedy, podľa tohto sme videli, ktoré triedy sú najčastejšie chybné a na ktorých sa sieť najčastejšie mýli.
